### DSL generation

In [38]:
#
# Save DSL as files
#
class DSLSaver: 
    def save(self, dsl_file_path, dsl):
        with open(dsl_file_path, 'w') as dsl_file:
            dsl_file.truncate(0)
            dsl_file.write(dsl)
        dsl_file.close()

In [34]:
#
# Utilities to generate various DSL files
#
import random
from os.path import basename

# path to save the generated data
save_path = "data_without_css/"

# function that opens file and returns the text inside it
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

# tokens to use for data generation
header_tokens = ["header", "header-right"]
grid_tokens = ["single", "double", "quadruple"]
header_content_tokens = ["btn-active", "btn-inactive", "btn-active", "btn-inactive", "empty"]
content_tokens = ["img", "empty", "empty", "empty"]
content2_tokens = ["btn", "big-title", "small-title", "big-text", "text", "big-title, text, btn", "big-title, text", "small-title, btn", "big-text, btn", "big-text, text, btn", "big-text, text", "empty"]

#
# grid utilities
#

# returns single or double or quadruple with the given content inside
def grid(i, content):
    if i == 1:
        result = "single {\n" + content + "\n}\n"
    elif i == 2:
        result = "double {\n" + content + "\n}\n"
    elif i == 4:
        result = "quadruple {\n" + content + "\n}\n"
    return result

# various types of grids
# s
# d d
# d q q
# q d q
# q q d
# q q q q

# 1st type of grid: 1 single
def grid1(contents):
    return grid(1, contents[0])
# 2nd type of grid: 2 doubles
def grid2(contents):
    return grid(2, contents[0]) + grid(2, contents[1])
# 3rd type of grid: 1 double and 2 quadraples
def grid3(index, contents):
    result = ""
    for i in range(3):
        if (i == index):
            result += grid(2, contents[i])
        else:
            result += grid(4, contents[i])
    return result
# 4th type of grid: 4 quadraples
def grid4(contents):
    result = ""
    for i in range(4):
        result += grid(4, contents[i])
    return result

# generates the type of grid according to the given index
def generate_grid(i, contents):
    if (i == 0):
        return grid1(contents)
    elif (i == 1):
        return grid2(contents)
    elif (i >= 2 and i < 5):
        return grid3(i - 2, contents)
    else:
        return grid4(contents)




In [ ]:
#
# Loop to actually generate various DSL files
#

# index of loop that will be used for filename
index = 0

# 2 kinds of header tokens (left aligned, right aligned)
for header_token in header_tokens:
    # 6 types of grid for the first row
    for i in range(6):
        # 6 types of grid for the second row
        for j in range(6):
            # 6 types of grid for the header
            for k in range(6):
                # 5 variations of the content to go inside each row
                for z in range(5):
                    contents = []
                    contents2 = []
                    header_contents = []
                    dsl = ""
                    
                    # content generation
                    header_contents = [random.choice(header_content_tokens) for _ in range(k + 1)]
                    contents = random.sample( content_tokens, len(content_tokens) )
                    contents2 = [random.choice(content2_tokens) for _ in range(j + 1)]

                    # header
                    dsl += header_token + " {\n"
                    dsl += generate_grid(k, header_contents)
                    dsl += "}\n"


                    # container begin
                    dsl += "container {\n"

                    # first row
                    dsl += "row {\n"
                    dsl += generate_grid(i, contents)
                    dsl += "}\n"

                    # second row
                    dsl += "row {\n"
                    dsl += generate_grid(j, contents2) 
                    dsl += "}\n"

                    # container end
                    dsl += "}\n"

                    
                    # save dsl file
                    dsl_file_path = "{}{:04d}.gui".format(save_path, index)
                    dsl_saver = DSLSaver()
                    dsl_saver.save(dsl_file_path, dsl)

                    index += 1



### DSL to HTML compilation

In [40]:
#!/usr/bin/env python
#
# DSL to HTML Compiler
#

import json
from classes.Node import *
from classes.Utils import *

# Generate random text and return the leaf node with the generated text inside it
def render_content_with_text(key, value):
    if FILL_WITH_RANDOM_TEXT:
        if key.find("btn") != -1:
            value = value.replace(TEXT_PLACE_HOLDER, Utils.get_random_text())
        elif key.find("title") != -1:
            value = value.replace(TEXT_PLACE_HOLDER, Utils.get_random_text(length_text=5, space_number=0))
        elif key.find("text") != -1:
            value = value.replace(TEXT_PLACE_HOLDER,
                                  Utils.get_random_text(length_text=56, space_number=7, with_upper_case=False))
        elif key.find("img") != -1:
            value = value.replace(TEXT_PLACE_HOLDER, image_url)
        elif key.find("empty") != -1:
            value = ""
            
    return value


# DSL to HTML compiler originally written by Tony Beltramelli - www.tonybeltramelli.com (pix2code)
class Compiler:
    def __init__(self, dsl_mapping_file_path):
        # load the DSL syntax file
        with open(dsl_mapping_file_path) as data_file:
            self.dsl_mapping = json.load(data_file)
            
        # save some tokens as class instance variables
        self.opening_tag = self.dsl_mapping["opening-tag"]
        self.closing_tag = self.dsl_mapping["closing-tag"]
        self.css_file_name = self.dsl_mapping["css-file-name"]
        self.content_holder = self.opening_tag + self.closing_tag

        # start the HTML hierarchy with the root node -- body
        self.root = Node("body", None, self.content_holder)

    def compile(self, tokens, output_file_path, css_file_path):
        
        # read the given tokens and parse them into a list
        dsl_file = tokens.copy()
        
        dsl_file = dsl_file[1:-1]
        dsl_file = ' '.join(dsl_file)
        
        dsl_file = dsl_file.replace('{', '{8').replace('}', '8}8')
        dsl_file = dsl_file.replace(' ', '')
        dsl_file = dsl_file.split('8')
        dsl_file = list(filter(None, dsl_file))
        
        # loop starting from the root (body element)
        current_parent = self.root
        for token in dsl_file:
            token = token.replace(" ", "").replace("\n", "")
           
            if token.find(self.opening_tag) != -1:
                token = token.replace(self.opening_tag, "")

                element = Node(token, current_parent, self.content_holder)
                current_parent.add_child(element)
                current_parent = element
            elif token.find(self.closing_tag) != -1:
                current_parent = current_parent.parent
            else:
                tokens = token.split(",")
                for t in tokens:
                    element = Node(t, current_parent, self.content_holder)
                    current_parent.add_child(element)

        output_html = self.root.render(self.dsl_mapping, rendering_function=render_content_with_text)
        
        if output_html is None:
            return "Parsing Error"
        
        with open(output_file_path, 'w') as output_file:
            output_file.truncate(0)
            output_file.write(output_html)
            
        output_file.close()
        return output_html
    
    
FILL_WITH_RANDOM_TEXT = True
TEXT_PLACE_HOLDER = "[]"
image_url = "images/blue.png"


In [43]:
#
# Compile the pix2code DSL files into HTML files
#

dsl_path = "web-dsl-mapping.json"
save_path = "data_without_css/"

# loop through 2160 different DSL files
for index in range(2160):
    # get the input DSL file
    input_file = "{}{:04d}.gui".format(save_path, index)
    
    # process the input into a string form
    file_uid = basename(input_file)[:basename(input_file).find(".")]
    path = input_file[:input_file.find(file_uid)]
    input_file_path = "{}{}.gui".format(path, file_uid)
    input_data = '<START> ' + load_doc(input_file_path) + ' <END>'
    
    # Add a space after each comma
    input_data = input_data.replace(',', ' ,')
    
    # Seperate all the words with a single space
    input_data = input_data.split()
    
    # Set the output file path as the same name of input file but with file extension of HTML
    output_file_path = "{}{:04d}.html".format(save_path, index)
    css_file_path = "{}{:04d}.css".format(save_path, index)

    # Compile the array that contains input DSL tokens into HTML files
    compiler = Compiler(dsl_path)
    compiler.compile(input_data, output_file_path, css_file_path)

### Compile the pix2code DSL files into HTML files

In [54]:

import os
for input_file in os.listdir("pix2code"):
    if input_file.endswith(".gui"):
        file_uid = basename(input_file)[:basename(input_file).find(".")]
        input_file_path = "pix2code/{}.gui".format(file_uid)
        input_data = '<START> ' + load_doc(input_file_path) + ' <END>'
        
        # Add a space after each comma
        input_data = input_data.replace(',', ' ,')
        
        # Seperate all the words with a single space
        input_data = input_data.split()
        
        output_file_path = "pix2code/{}.html".format(file_uid)
        css_file_path = "pix2code/{}.css".format(file_uid)
        
        compiler = Compiler(dsl_path)
        compiler.compile(input_data, output_file_path, css_file_path)

### ETC

In [ ]:
# compare the original pix2code HTML with the modified HTML
print("<html>\n  <head>\n   <meta charset=\"utf-8\">\n    <meta name=\"viewport\" content=\"width=device-width, initial-scale=1\">\n    <link rel=\"stylesheet\" href=\"https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta3/dist/css/bootstrap.min.css\" rel=\"stylesheet\" integrity=\"sha384-eOJMYsd53ii+scO/bJGFsiCZc+5NDVN2yr8+0RDqr0Ql0h+rP48ckxlpbzKgwra6\" crossorigin=\"anonymous\">\n    <link rel=\"stylesheet\" href=\"styles/common.css\" type=\"text/css\">\n    <link rel=\"stylesheet\" href=\"+++\" type=\"text/css\">\n    <title>Scaffold</title>\n  </head>\n  <body>\n    <aside></aside>\n      {}\n    <script src=\"js/jquery.min.js\"></script>\n    <script src=\"js/bootstrap.min.js\"></script>\n  </body>\n</html>\n")
print("<html>\n  <header>\n    <meta charset=\"utf-8\">\n    <meta name=\"viewport\" content=\"width=device-width, initial-scale=1\">\n    <link rel=\"stylesheet\" href=\"https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css\" integrity=\"sha384-BVYiiSIFeK1dGmJRAkycuHAHRg32OmUcww7on3RYdg4Va+PmSTsz/K68vbdEjh4u\" crossorigin=\"anonymous\">\n<link rel=\"stylesheet\" href=\"https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap-theme.min.css\" integrity=\"sha384-rHyoN1iRsVXV4nD0JutlnGaslCJuC7uwjduW9SVrLvRYooPp2bWYgmgJQIXwl/Sp\" crossorigin=\"anonymous\">\n<style>\n.header{margin:20px 0}nav ul.nav-pills li{background-color:#333;border-radius:4px;margin-right:10px}.col-lg-3{width:24%;margin-right:1.333333%}.col-lg-6{width:49%;margin-right:2%}.col-lg-12,.col-lg-3,.col-lg-6{margin-bottom:20px;border-radius:6px;background-color:#f5f5f5;padding:20px}.row .col-lg-3:last-child,.row .col-lg-6:last-child{margin-right:0}footer{padding:20px 0;text-align:center;border-top:1px solid #bbb}\n</style>\n    <title>Scaffold</title>\n  </header>\n  <body>\n    <main class=\"container\">\n      {}\n      <footer class=\"footer\">\n        <p>&copy; Tony Beltramelli 2017</p>\n      </footer>\n    </main>\n    <script src=\"js/jquery.min.js\"></script>\n    <script src=\"js/bootstrap.min.js\"></script>\n  </body>\n</html>\n",)